In [1]:
import sys
sys.path.append('/srv/share3/hagrawal9/project/sokoban/habitat-sim/')
sys.path.append('/srv/share3/hagrawal9/project/sokoban/habitat-api/')

In [2]:
cd ..

/coc/pskynet3/hagrawal9/project/sokoban/habitat-api


In [7]:
import gzip
import habitat
import habitat_sim
import habitat_sim.bindings as hsim
import json
import matplotlib.pyplot as plt
import magnum as mn
import numpy as np
import os
import pandas as pd
import tqdm 

from habitat.config import Config
from examples.run_ortools import *
from habitat_sim.physics import MotionType
from habitat_sim.attributes import PhysicsObjectAttributes
from habitat_sim.helper import *
from PIL import Image
from typing import Any, Dict, List, Optional

%matplotlib inline

In [8]:
train_df = pd.read_pickle('data/sokoban_gibson_train.pkl')

In [9]:
train_df

,id,name,split_full,split_full+,split_medium,split_tiny,area,floor,navigation_complexity,room,ssa
441,Roeville,model-304,train,train,none,none,484.984,4.0,1.030,21.0,1.008
455,Sasakwa,model-144,train,train,none,none,45.350,1.0,2.543,5.0,1.134
484,Spencerville,model-471,train,train,none,none,72.589,1.0,1.768,4.0,1.149
355,Mifflintown,model-531,train,train,none,none,165.089,2.0,2.151,28.0,1.340
332,Maryhill,model-307,train,train,none,none,531.583,4.0,2.939,12.0,1.406
496,Stokes,model-198,train,train,none,none,276.521,3.0,4.085,9.0,1.455
134,Crandon,model-112,train,train,none,none,298.350,3.0,3.632,13.0,1.609
414,Placida,model-398,train,train,none,none,83.857,1.0,1.407,8.0,1.764
488,Springhill,model-43,train,train,none,none,549.540,4.0,1.388,11.0,1.896
411,Pettigrew,model-525,train,train,none,none,161.586,1.0,2.456,11.0,2.061


In [4]:
settings = {
    "max_frames": 10,
    "width": 256,  # Spatial resolution of the observations
    "height": 256,
    "scene": 'data/scene_datasets/coda/coda.glb',  # Scene path
    # "scene": 'data/scene_datasets/habitat-test-scenes/skokloster-castle.glb',
    # "scene": 'data/scene_datasets/gibson/Mobridge.glb',
    "default_agent": 0,
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": False,  # Semantic sensor
    "depth_sensor": True,  # Depth sensor
    "seed": 1,
    "enable_physics": True,
    "physics_config_file": "data/default.phys_scene_config.json", 
    "silent": False, 
    "num_objects": 10,
    "compute_shortest_path": False,
    "compute_action_shortest_path": False,
    "save_png": True,
    "render_to_ui": False
}

In [5]:
cfg = make_cfg(settings)
sim = habitat_sim.Simulator(cfg)
agent_id = settings["default_agent"]
shortest_path = None

Initializing Agents
Initializing Objects


In [6]:
navmesh_settings = habitat_sim.NavMeshSettings()
navmesh_settings.set_defaults()
navmesh_settings.agent_radius = 0.2
navmesh_settings.agent_height = 0.5
navmesh_settings.agent_max_climb = 0.1
# navmesh_settings.agent_max_slope = 0.1

In [7]:
sim.recompute_navmesh(sim.pathfinder, navmesh_settings, False)

True

In [8]:
def get_rotation(oid):
    quat = sim.get_rotation(0)
    return np.array(quat.vector).tolist() + [quat.scalar]

In [9]:
def init_episode_dict(episode_num):
    episode_dict = {
        'episode_id': episode_num, 
        'scene_id': 'data/scene_datasets/coda/coda.glb',
        'start_position': np.array(sim.get_translation(0)).tolist(), 
        'start_rotation': get_rotation(0), 
        'info': {}, 
        'object_templates': [
            {
                'object_key': 'sphere_blue', 
                'object_template': 'data/test_assets/objects/sphere_blue.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_red', 
                'object_template': 'data/test_assets/objects/sphere.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_pink', 
                'object_template': 'data/test_assets/objects/sphere_pink.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_maroon', 
                'object_template': 'data/test_assets/objects/sphere_maroon.glb', 
                # other stats like scale etc that might be important. 
            },
            {
                'object_key': 'sphere_purple', 
                'object_template': 'data/test_assets/objects/sphere_purple.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_yellow', 
                'object_template': 'data/test_assets/objects/sphere_yellow.glb', 
                # other stats like scale etc that might be important. 
            },
            {
                'object_key': 'sphere_orange', 
                'object_template': 'data/test_assets/objects/sphere_orange.glb', 
                # other stats like scale etc that might be important. 
            },
            {
                'object_key': 'sphere_green', 
                'object_template': 'data/test_assets/objects/sphere_green.glb', 
                # other stats like scale etc that might be important. 
            }
        ],
        'objects': [

        ],
        'goals': [ 
            
        ], 
    }
    return episode_dict

def add_object_details(episode_dict, num_objects):
    for i in range(1, num_objects + 1):
        object_template = {
            'object_id': i, 
            'object_key': i%8,
            'position': np.array(sim.get_translation(i)).tolist(), 
            'rotation': get_rotation(i),
        }
        episode_dict['objects'].append(object_template)
    
    return episode_dict

def add_goal_details(episode_dict, num_objects):
    for i in range(1 + num_objects, num_objects*2 + 1):
        goal_template = {
            'position': np.array(sim.get_translation(sim.get_existing_object_ids()[i])).tolist(), 
            'rotation': get_rotation(sim.get_existing_object_ids()[i]),
        }
            
        episode_dict['goals'].append(goal_template)
    return episode_dict

# set the number of objects to 1 always for now. 
def build_episode(episode_num, num_objects=1):
    episodes = {'episodes': []}
    for episode in tqdm.tqdm(range(episode_num)):
        start_state = init_agent_state(sim, navmesh_settings, agent_id)
        sim.recompute_navmesh(sim.pathfinder, navmesh_settings, True)
        init_test_scene(sim, navmesh_settings, num_objects * 2)
        sim.recompute_navmesh(sim.pathfinder, navmesh_settings, True)
        
        episode_dict = init_episode_dict(episode)
        num_object = np.random.choice(range(num_objects)) + 1
        episode_dict = add_object_details(episode_dict, num_object)
        episode_dict = add_goal_details(episode_dict, num_object)
        episodes['episodes'].append(episode_dict)
        
        
    
    return episodes

In [12]:
episodes = build_episode(1000, num_objects=5)

100%|██████████| 1000/1000 [21:40<00:00,  1.30s/it]


In [13]:
with gzip.open('data/datasets/sokoban/coda/v1/train/content/object_5_scene_1000_coda.json.gz', "wt") as f:
    json.dump(episodes, f)

In [16]:
episodes = build_episode(10, num_objects=5)
with gzip.open('data/datasets/sokoban/coda/v1/train/content/val_object_5_scene_10_coda.json.gz', "wt") as f:
    json.dump(episodes, f)

100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


In [15]:
episodes = build_episode(100, num_objects=5)
with gzip.open('data/datasets/sokoban/coda/v1/train/content/test_object_5_scene_100_coda.json.gz', "wt") as f:
    json.dump(episodes, f)

100%|██████████| 100/100 [02:05<00:00,  1.25s/it]
